<a href="https://colab.research.google.com/github/cristiandarioortegayubro/BDS/blob/main/modulo.04/bds_optimizacion_005_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>


<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20Scikit-learn.png?raw=true">
</p>


 # **<font color="DeepPink">Ajuste de hiperparámetros por grid-search</font>**

<p align="justify">
♥ En este Colab, mostraremos cómo optimizar los hiperparámetros utilizando un enfoque de búsqueda en cuadrícula (grid-search).

 ## **<font color="DeepPink">Carga de las librerías</font>**

In [1]:
import numpy as np
import pandas as pd

In [2]:
import plotly.express as px

 ## **<font color="DeepPink">Carga del conjunto de datos</font>**

In [3]:
adult_census = pd.read_csv("https://raw.githubusercontent.com/cristiandarioortegayubro/BDS/main/datasets/adult_census.csv")
adult_census

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,class
0,25,Private,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


 ## **<font color="DeepPink">Separamos la variable objetivo y las variables explicativas</font>**

<p align="justify">
👀 Asignamos a un objeto la variable objetivo:
</p>


In [4]:
target_name = "class"
y = adult_census[target_name]
y

0         <=50K
1         <=50K
2          >50K
3          >50K
4         <=50K
          ...  
48837     <=50K
48838      >50K
48839     <=50K
48840     <=50K
48841      >50K
Name: class, Length: 48842, dtype: object

<p align="justify">
👀 Eliminamos de nuestros datos la variable objetivo y la columna <code>education-num</code> que duplica la información de la columna <code>education</code>.

In [5]:
X = adult_census.drop(columns=[target_name, "education-num"])
X.head()

,age,workclass,education,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,25,Private,11th,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States
1,38,Private,HS-grad,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States
2,28,Local-gov,Assoc-acdm,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States
3,44,Private,Some-college,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States
4,18,?,Some-college,Never-married,?,Own-child,White,Female,0,0,30,United-States


 ## **<font color="DeepPink">Conjunto de entrenamiento y conjunto de prueba</font>**

👀 Dividimos en conjunto de entrenamiento y prueba

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

 ## **<font color="DeepPink">Pipeline</font>**

<p align="justify">
✅ Definiremos un Pipeline que va a manejar características numéricas como categóricas. El primer paso es seleccionar todas las columnas categóricas...

In [8]:
from sklearn.compose import make_column_selector as selector

In [9]:
categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(X)

👀 Vemos las columnas categóricas...

In [10]:
categorical_columns

['workclass',
 'education',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native-country']

<p align="justify">
Usaremos un modelo basado en árboles de decisión (<code>HistGradientBoostingClassifier</code>). Considerando los siguientes supuestos:

* Las variables numéricas no necesitan escala;
* Las variables categóricas se pueden tratar con un `OrdinalEncoder`.

🛑 En general, los modelos basados en árboles pueden manejar directamente variables categóricas codificadas ordinalmente, ya que realizan divisiones binarias en las variables para construir el árbol. Sin embargo, esto no significa que siempre debas usar `OrdinalEncoder` para variables categóricas en modelos basados en árboles. Si hay un gran número de clases o si la codificación ordinal no tiene un orden significativo, puede ser preferible utilizar otras técnicas de codificación o incluso considerar `OneHotEncoder`.



<p align="justify">
✅ Ahora construimos nuestro <code>OrdinalEncoder</code>.

In [11]:
from sklearn.preprocessing import OrdinalEncoder

In [12]:
categorical_preprocessor = OrdinalEncoder(handle_unknown="use_encoded_value",
                                          unknown_value=-1)

<p align="justify">
<code>handle_unknown="use_encoded_value"</code> indica que se utilizará un valor de codificación especial para las clases desconocidas. En este caso, el valor especificado en <code>unknown_value</code> se utilizará para codificar las clases desconocidas. Es decir, cuando se encuentra una clase desconocida en los datos de prueba, se le asignará el valor de -1 en la codificación resultante.

In [13]:
categorical_preprocessor

OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)

<p align="justify">
✅ Luego usamos un <code>ColumnTransformer</code> para seleccionar las columnas categóricas y les aplicamos el <code>OrdinalEncoder</code>

In [14]:
from sklearn.compose import ColumnTransformer

In [15]:
preprocessor = ColumnTransformer([
    ('cat_preprocessor', categorical_preprocessor, categorical_columns)],
    remainder='passthrough')

In [16]:
preprocessor

ColumnTransformer(remainder='passthrough',
                  transformers=[('cat_preprocessor',
                                 OrdinalEncoder(handle_unknown='use_encoded_value',
                                                unknown_value=-1),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

<p align="justify">
✅ Finalmente, usamos un clasificador basado en árboles (histogram gradient-boosting) para predecir si una persona gana o no más de 50 k$ al año.

[Documentación](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.HistGradientBoostingClassifier.html)

In [17]:
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.pipeline import Pipeline

In [18]:
model = Pipeline([("preprocessor", preprocessor),
                  ("classifier", HistGradientBoostingClassifier(random_state=42))])

In [19]:
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('cat_preprocessor',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier',
                 HistGradientBoostingClassifier(random_state=42))])

 # **<font color="DeepPink">Ajuste mediante grid-search</font>**

<p align="justify">
En el colab anterior, usamos <code>for</code> para encontrar valores óptimos de los parámetros.
<br><br>
<code>GridSearchCV</code> es una clase de <code>scikit-learn</code> que implementa una lógica muy similar con un código más simple.
Veamos cómo usar <code>GridSearchCV</code> para realizar dicha búsqueda. Dado que la búsqueda será costosa, solo exploraremos la combinación de <b>tasa de aprendizaje</b> y el <b>número máximo de nodos terminales (hojas)</b> en cada árbol.
<br><br>
- <code>learning_rate</code>: controla qué tan rápido se ajusta un modelo durante el proceso de entrenamiento. Por ejemplo, una tasa de aprendizaje más alta (como 0.1) implica ajustes más grandes y rápidos, lo que puede conducir a una convergencia más rápida pero también a un riesgo de sobreajuste. Por otro lado, una tasa de aprendizaje más baja (como 0.01) implica ajustes más pequeños y graduales, lo que puede llevar más tiempo pero también reducir el riesgo sobreajuste. La elección de la tasa de aprendizaje depende de encontrar un equilibrio entre un aprendizaje rápido y una buena generalización a nuevos datos. Por defecto es 0.1.
<br><br>
- <code>max_leaf_nodes</code>: número máximo de nodos hoja para cada árbol. Debe ser estrictamente mayor que 1. Por defecto no hay límite máximo.
<br><br>
<b>Grid-search</b> es una técnica utilizada para encontrar la combinación óptima de hiperparámetros para un modelo de aprendizaje automático. Los hiperparámetros afectan el rendimiento y la capacidad del modelo. Esta técnica implica especificar una cuadrícula de valores posibles para diferentes hiperparámetros y luego evaluar el rendimiento del modelo en cada combinación de valores. Esto implica entrenar y evaluar el modelo con todas las posibles combinaciones de hiperparámetros.

In [20]:
from sklearn.model_selection import GridSearchCV

In [21]:
param_grid = {
    'classifier__learning_rate': [0.01, 0.1, 1, 10], #corchete o parentesis
    'classifier__max_leaf_nodes': (3, 10, 30)
    }

In [22]:
model_grid_search = GridSearchCV(model, param_grid=param_grid, n_jobs=2)
model_grid_search.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('cat_preprocessor',
                                                                         OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                                        unknown_value=-1),
                                                                         ['workclass',
                                                                          'education',
                                                                          'marital-status',
                                                                          'occupation',
                                                                          'relationship',
                                                                          'race',
                                                                          'sex',
                                                                          'native-country'])])),
                                       ('classifier',
                                        HistGradientBoostingClassifier(random_state=42))]),
             n_jobs=2,
             param_grid={'classifier__learning_rate': [0.01, 0.1, 1, 10],
                         'classifier__max_leaf_nodes': (3, 10, 30)})

<p align="justify">
<code>GridSearchCV</code> usa <code>param_grid</code> que define todos los hiperparámetros y sus valores asociados. El <code>grid-search</code> se encargará de crear todas las combinaciones posibles y probarlas.
<br><br>
El número de combinaciones será igual al producto del número de valores a explorar para cada parámetro (por ejemplo, en nuestro ejemplo, combinaciones $4 × 3$). Por lo tanto, agregar nuevos parámetros con sus valores asociados para ser explorados.
<br><br>
Una vez que se ajusta grid-search, se puede usar como cualquier otro predictor llamando a <code>predict</code> y <code>predict_proba</code>. Internamente, utilizará el modelo con los mejores parámetros encontrados durante el <code>fit</code>.
<br><br>
Ahora vamos a obtener las predicciones y el <code>accuracy</code> del modelo usando el estimador con los mejores parámetros.

In [23]:
model_grid_search.predict(X_test)

array([' <=50K', ' <=50K', ' >50K', ..., ' <=50K', ' <=50K', ' >50K'],
      dtype=object)

In [24]:
model_grid_search.score(X_test, y_test)

0.8789615920072066

<p align="justify">
👀 Puede conocer estos parámetros mirando el atributo <code>best_params_</code>.

In [25]:
print("")
print(f"El mejor conjunto de parámetros es: "
      f"{model_grid_search.best_params_}")


El mejor conjunto de parámetros es: {'classifier__learning_rate': 0.1, 'classifier__max_leaf_nodes': 30}


<p align="justify">
Además del <code>accuracy</code>, podemos inspeccionar todos los resultados que se almacenan en el atributo <code>cv_results_</code> de la búsqueda en grid-search. Filtraremos algunas columnas específicas de estos resultados.

In [26]:
cv_results = pd.DataFrame(model_grid_search.cv_results_).sort_values("mean_test_score", ascending=False)
cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__learning_rate,param_classifier__max_leaf_nodes,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
5,1.934956,0.465501,0.198821,0.042148,0.1,30,"{'classifier__learning_rate': 0.1, 'classifier...",0.869387,0.872099,0.873464,0.869369,0.870734,0.871011,0.001588,1
4,0.978310,0.011626,0.133251,0.002882,0.1,10,"{'classifier__learning_rate': 0.1, 'classifier...",0.865702,0.871826,0.870325,0.864182,0.870188,0.868445,0.002956,2
7,0.451025,0.128743,0.088731,0.018592,1,10,"{'classifier__learning_rate': 1, 'classifier__...",0.861744,0.868550,0.865001,0.858449,0.863500,0.863449,0.003357,3
6,0.480056,0.162585,0.081700,0.015390,1,3,"{'classifier__learning_rate': 1, 'classifier__...",0.865566,0.860087,0.868550,0.862408,0.848621,0.861047,0.006839,4
8,0.377877,0.041804,0.106135,0.069280,1,30,"{'classifier__learning_rate': 1, 'classifier__...",0.859697,0.856129,0.864728,0.853399,0.859814,0.858753,0.003828,5
3,0.722323,0.022438,0.108531,0.002211,0.1,3,"{'classifier__learning_rate': 0.1, 'classifier...",0.852191,0.855992,0.857767,0.852853,0.854081,0.854577,0.002054,6
2,1.957173,0.500624,0.160723,0.007601,0.01,30,"{'classifier__learning_rate': 0.01, 'classifie...",0.848096,0.843571,0.846983,0.846164,0.848075,0.846578,0.001669,7
1,1.103134,0.143980,0.186500,0.058569,0.01,10,"{'classifier__learning_rate': 0.01, 'classifie...",0.822028,0.816544,0.817909,0.815861,0.817636,0.817996,0.002148,8
0,0.742330,0.020428,0.114997,0.006167,0.01,3,"{'classifier__learning_rate': 0.01, 'classifie...",0.800191,0.795932,0.794704,0.796478,0.796069,0.796675,0.001855,9
9,0.346765,0.125224,0.061573,0.010667,10,3,"{'classifier__learning_rate': 10, 'classifier_...",0.284564,0.279825,0.275594,0.288834,0.288561,0.283476,0.005123,10


<p align="justify">
Con solo $2$ parámetros, es posible que queramos visualizar grid-search como un <b>mapa de calor</b>. Necesitamos transformar nuestro <code>cv_results</code> en un <code>DataFrame</code> donde:

- Las filas corresponderán a los valores de la tasa de aprendizaje;
- Las columnas corresponderán al número máximo de hojas;
- El contenido del <code>DataFrame</code> serán las puntuaciones medias de las pruebas.

In [27]:
pivoted_cv_results = cv_results.pivot_table(values="mean_test_score",
                                            index=["param_classifier__learning_rate"],
                                            columns=["param_classifier__max_leaf_nodes"]
                                            )

In [28]:
pivoted_cv_results

param_classifier__max_leaf_nodes,3,10,30
param_classifier__learning_rate,,,
0.01,0.796675,0.817996,0.846578
0.10,0.854577,0.868445,0.871011
1.00,0.861047,0.863449,0.858753
10.00,0.283476,0.257241,0.264912


👀 Generando el mapa de calor

In [34]:
fig = px.imshow(pivoted_cv_results.round(2),
                x=["3", "10", "30"],
                y=["0.01", "0.10", "1.00", "10.00"],
                text_auto=True,
                aspect="auto",
                title="Mapa de Calor",
                labels=dict(x="Max leaf nodes",
                            y="Learning rate",
                            color="Mean Test Score"))

fig.show()

<p align="justify">
📊 El mapa de calor destaca lo siguiente:

- Para valores demasiado altos de `learning_rate`, el rendimiento de generalización del modelo está degradado y no se puede corregir ajustando con  `max_leaf_nodes`.
- Se observa que la elección óptima de `max_leaf_nodes` depende del valor de `learning_rate`.
- En particular, se observa una "diagonal" de buenos modelos con una `accuracy` cercana a 0.87: cuando el valor de `max_leaf_nodes` es  aumentado, se debe disminuir el valor de `learning_rate`, para conservar un buen rendimiento.
<br>
<p align="justify">
🛑 El significado preciso de esos dos parámetros se explicará más adelante.
<br><br>
Por ahora notaremos que, en general, <b>no existe un único parámetro óptimo</b>, 2 modelos de las 12 configuraciones de parámetros alcanzan el máximo rendimiento (0.87).

 # **<font color="DeepPink">Conclusiones</font>**

<p align="justify">
👀 En este colab nosotros:
<br><br>
✅ Cargamos los datos de un archivo <code>CSV</code> usando <code>Pandas</code>.<br>
✅ Generamos un Pipeline.
<br>  
✅ Separamos en un conjunto de entrenamiento y un conjunto de prueba.
<br>
✅ Optimizamos los hiperparámetros de un modelo predictivo a través del grid-search.
<br>
✅ Vimos que una búsqueda en grid-search no necesariamente encuentra una solución óptima.



<br>
<br>
<p align="center"><b>
💗
<font color="DeepPink">
Hemos llegado al final de nuestro colab, a seguir codeando...
</font>
</p>
<br>
<p align="center">
<img src="https://github.com/cristiandarioortegayubro/BDS/blob/main/images/Logo%20BDS%20Horizontal%208.png?raw=true">
</p>

---
